In [33]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf

In [ ]:
from pypdf import PdfReader
import pikepdf 
import os 
import shutil

# Paths das pastas
input_folder = "/Users/usuario-rtd/Documents/mapa_regex_gb_v1/source_maps"
output_folder = "/Users/usuario-rtd/Documents/mapa_regex_gb_v1/decrypted_source_maps"
encrypted_folder = "/Users/usuario-rtd/Documents/mapa_regex_gb_v1/source_maps/encrypted"

# Verificar se as pastas de saída e de encriptados existem, caso contrário, criá-las
os.makedirs(output_folder, exist_ok=True)
os.makedirs(encrypted_folder, exist_ok=True)

# Processar todos os arquivos PDF no diretório de entrada
for file_name in os.listdir(input_folder):
    if file_name.lower().endswith(".pdf"):  # Verifica se é um arquivo PDF
        input_pdf_path = os.path.join(input_folder, file_name)
        output_pdf_path = os.path.join(output_folder, file_name)
        encrypted_pdf_path = os.path.join(encrypted_folder, file_name)

        try:
            # Tenta abrir e salvar o PDF sem senha
            with pikepdf.open(input_pdf_path) as pdf:
                pdf.save(output_pdf_path)
            print(f"PDF '{file_name}' desbloqueado com sucesso! Salvo em: {output_pdf_path}")
        except pikepdf.PasswordError:
            print(f"O PDF '{file_name}' está protegido por senha e não pode ser desbloqueado sem ela.")
        except FileNotFoundError as e:
            print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
        except Exception as e:
            print(f"Ocorreu um erro ao processar '{file_name}': {e}")
        finally:
            # Mover o arquivo original (encryptado) para a pasta de encriptados
            shutil.move(input_pdf_path, encrypted_pdf_path)
            print(f"PDF '{file_name}' movido para a pasta de encriptados: {encrypted_pdf_path}")

In [74]:
from PyPDF2 import PdfReader
import os
import shutil
import re

# Paths das pastas
input_folder = "/Users/usuario-rtd/Documents/mapa_regex_gb_v1/decrypted_source_maps"
processed_folder = "/Users/usuario-rtd/Documents/mapa_regex_gb_v1/decrypted_source_maps/processed"

# Verificar se a pasta de processados existe, caso contrário, criá-la
os.makedirs(processed_folder, exist_ok=True)

# Obter a lista de PDFs na pasta de entrada
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

if pdf_files:
    # Processar apenas o primeiro PDF
    file_name = pdf_files[0]
    input_pdf_path = os.path.join(input_folder, file_name)
    processed_pdf_path = os.path.join(processed_folder, file_name)
    
    try:
        # Inicializar o texto completo para o PDF atual
        full_text = ""
        
        # Ler o PDF
        reader = PdfReader(input_pdf_path)
        
        # Iterar sobre as páginas e adicionar os delimitadores
        for idx, page in enumerate(reader.pages):
            full_text += f"------------------ PAGINA {idx} ----------------\n"
            full_text += page.extract_text()
        
        # Dividir o texto baseado nos delimitadores
        paginas = re.split(r"------------------ PAGINA \d+ ----------------", full_text)
        paginas = [pagina.strip() for pagina in paginas if pagina.strip()]
        
        # Exibir a quantidade de páginas úteis detectadas
        print(f"\nTotal de páginas detectadas em '{file_name}': {len(paginas)}\n")
        
        # Criar dicionário para armazenar o conteúdo das páginas
        texto_paginas = {}
        for idx, pagina in enumerate(paginas, start=1):
            texto_paginas[f"texto_pagina{idx}"] = pagina
        
        # Imprimir o conteúdo das páginas (limitar visualização para validação)
        for key, value in texto_paginas.items():
            print(f"--- {key} ---")
            print(value[:1000])  # Limitar a 1000 caracteres para evitar poluição visual
            print("\n")
        
    except Exception as e:
        print(f"Ocorreu um erro ao processar '{file_name}': {e}")
    finally:
        # Mover o PDF processado para a pasta de processados
        shutil.move(input_pdf_path, processed_pdf_path)
        print(f"PDF '{file_name}' movido para a pasta de processados: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")


Total de páginas detectadas em 'decrypted_my_pdf.pdf': 5

--- texto_pagina1 ---
Informação comunicada pela instituição: BANCO SANTANDER TOTTA, S.A. (0018)
Montantes
Total em dívida
do qual, em incumprimento
2 974,16 €
0,00 €
Nº devedores no contrato
2
Início
2018-03-06
Tipo de negociação
Totalmente nova
Produto financeiro
Crédito pessoal 
Tipo de responsabilidade
Devedor
Fim
2027-12-06
Em litígio judicial
Não
Vencido
0,00 €
Entrada incumpr.
Não Aplicável
Tipo 
Valor
Número
-
-
-
Garantias
Abatido ao ativo
92,20 €
0,00 €
Potencial
0,00 €
Prestação 
Periodicidade
Mensal
Montantes
Total em dívida
do qual, em incumprimento
34 781,66 €
0,00 €
Nº devedores no contrato
2
Início
2007-07-30
Tipo de negociação
Totalmente nova
Produto financeiro
Crédito à habitação
Tipo de responsabilidade
Devedor
Fim
2046-02-02
Em litígio judicial
Não
Vencido
0,00 €
Entrada incumpr.
Não Aplicável
Tipo 
Valor
Número
1410
64 541,02 €
1
-
-
-
Garantias
Abatido ao ativo
207,51 €
0,00 €
Potencial
0,00 €
Prestação 
P

In [ ]:
import re
import pandas as pd

# Regexes para capturar os elementos de interesse
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Montantes\s+Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s+\d+\s+([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})\s+Em litígio judicial"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})\s+Tipo")
}

# Função para aplicar regex em texto
def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None
 
# Inicializar lista para armazenar os dados
data = []

# Iterar sobre as páginas do dicionário texto_paginas
for page_number, page_text in texto_paginas.items():
    # Extrair informações de cada página usando regex
    row = {
        'pagina_pdf': page_number,
        'nome': get_feature(page_text, 'nome'),
        'nif': get_feature(page_text, 'nif'),
        'mes_mapa': get_feature(page_text, 'mes_mapa'),
        'instituicao': get_feature(page_text, 'instituicao'),
        'divida': get_feature(page_text, 'total_em_divida'),
        'litigio': get_feature(page_text, 'litigio'),
        'parcela': get_feature(page_text, 'abatido_ativo'),
        'garantias': get_feature(page_text, 'garantias'),
        'num_devedores': get_feature(page_text, 'num_devedores'),
        'prod_financeiro': get_feature(page_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(page_text,'entrada_incumpr'),
        'dat_inicio': get_feature(page_text, 'dat_inicio'),
        'dat_fim': get_feature(page_text, 'dat_fim')
    }
    # Adicionar os dados ao conjunto
    data.append(row)

# Criar o DataFrame
df = pd.DataFrame(data)
df

,pagina,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,texto_pagina1,LUIS MANUEL DA CONCEIÇÃO CLEMENTE QUINTINO,192380613,outubro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)","2 974,16",Não,"92,20","64 541,02",2,Crédito pessoal,None,2018-03-06,2027-12-06
1,texto_pagina2,LUIS MANUEL DA CONCEIÇÃO CLEMENTE QUINTINO,192380613,outubro de 2024,None,"41 244,33",Não,"257,71","82 795,32",2,Crédito à habitação,None,2003-12-23,2045-07-02
2,texto_pagina3,LUIS MANUEL DA CONCEIÇÃO CLEMENTE QUINTINO,192380613,outubro de 2024,"BANKINTER CONSUMER FINANCE, E.F.C., SA - SUCUR...","2 464,50",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2019-10-31,9999-12-31
3,texto_pagina4,LUIS MANUEL DA CONCEIÇÃO CLEMENTE QUINTINO,192380613,outubro de 2024,"UNICRE - INSTITUIÇÃO FINANCEIRA DE CRÉDITO, S....","21 461,11",Não,"330,29",None,1,Crédito pessoal,2024-09-30,2024-03-11,2034-03-11
4,texto_pagina5,LUIS MANUEL DA CONCEIÇÃO CLEMENTE QUINTINO,192380613,outubro de 2024,None,None,None,None,None,None,None,None,None,None
